# Real-World Motivation: Why Time-Series Validation Matters

## Realistic Synthetic Examples of Time-Series Challenges

---

**Purpose**: See how real-world financial/economic data behaves and why standard ML approaches fail catastrophically.

**Prerequisites**:
- [00_time_series_fundamentals.ipynb](00_time_series_fundamentals.ipynb) — ACF intuition, why shuffling fails

**What You'll Learn**:
1. How interest rates, stock returns, and unemployment actually behave
2. Why persistence makes prediction nearly impossible for some series
3. When you can and cannot add value over naive forecasts
4. How to diagnose your own data before building models

**Time**: ~30 minutes

---

**All data in this notebook is SYNTHETIC** — generated to mimic real-world patterns without using proprietary data. The patterns and lessons, however, are based on decades of empirical research.

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

from temporalcv.cv import WalkForwardCV
from temporalcv import compute_mase, compute_naive_error

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

def compute_acf(series, max_lag=20):
    """Compute autocorrelation function."""
    n = len(series)
    mean_s = np.mean(series)
    var_s = np.var(series)
    
    acf = []
    for lag in range(max_lag + 1):
        if lag == 0:
            acf.append(1.0)
        else:
            cov = np.mean((series[lag:] - mean_s) * (series[:-lag] - mean_s))
            acf.append(cov / var_s)
    return np.array(acf)

print("Setup complete. Let's explore real-world patterns.")

---

## Section 1: Treasury Rate Dynamics

**What we're mimicking**: 10-Year Treasury Yield (weekly data)

**Key characteristics**:
- Extremely high persistence (φ ≈ 0.995)
- Mean-reverting around a long-run equilibrium
- Small weekly changes (typically ±0.10%)

**The challenge**: If today's rate is 4.25%, tomorrow's rate is probably 4.24% or 4.26%. How do you beat "predict no change"?

In [ ]:
def generate_treasury_like(n_weeks=1040, phi=0.995, sigma=0.08, 
                           long_run_mean=2.5, seed=42):
    """
    Generate synthetic Treasury-like rate series.
    
    Mimics 10-year Treasury yields:
    - Very high persistence (φ ≈ 0.995)
    - Mean-reverting around long-run mean
    - Realistic weekly volatility (~8bp)
    
    Parameters
    ----------
    n_weeks : int
        Number of weekly observations (1040 ≈ 20 years)
    phi : float
        Autoregressive coefficient (persistence)
    sigma : float
        Innovation standard deviation
    long_run_mean : float
        Long-run equilibrium rate (%)
    seed : int
        Random seed for reproducibility
    """
    rng = np.random.default_rng(seed)
    rates = np.zeros(n_weeks)
    rates[0] = long_run_mean + rng.normal(0, sigma * 5)  # Start near equilibrium
    
    for t in range(1, n_weeks):
        # Ornstein-Uhlenbeck: mean-reverting AR(1)
        rates[t] = (phi * rates[t-1] + 
                    (1 - phi) * long_run_mean + 
                    sigma * rng.normal())
    
    return rates

# Generate 20 years of weekly Treasury-like rates
treasury_rates = generate_treasury_like(n_weeks=1040, phi=0.995, seed=42)

# Compute ACF
acf_treasury = compute_acf(treasury_rates, max_lag=52)  # 1 year of lags

print(f"Generated {len(treasury_rates)} weeks of synthetic Treasury rates")
print(f"Mean rate: {np.mean(treasury_rates):.2f}%")
print(f"Std dev: {np.std(treasury_rates):.2f}%")
print(f"ACF(1): {acf_treasury[1]:.4f}")
print(f"ACF(52) [1 year]: {acf_treasury[52]:.4f}")

In [ ]:
# Visualize Treasury dynamics
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Top left: Time series
ax = axes[0, 0]
weeks = np.arange(len(treasury_rates))
years = weeks / 52
ax.plot(years, treasury_rates, 'b-', linewidth=0.8)
ax.set_xlabel('Years')
ax.set_ylabel('Rate (%)')
ax.set_title('Synthetic 10-Year Treasury Yield (20 Years Weekly)', fontsize=11, fontweight='bold')
ax.axhline(y=np.mean(treasury_rates), color='red', linestyle='--', alpha=0.5, label='Long-run mean')
ax.legend()

# Top right: ACF
ax = axes[0, 1]
lags = np.arange(len(acf_treasury))
ax.bar(lags, acf_treasury, color='steelblue', alpha=0.7)
ax.axhline(y=0.9, color='red', linestyle='--', label='High persistence threshold')
ax.axhline(y=0, color='black', linewidth=0.5)
ax.set_xlabel('Lag (weeks)')
ax.set_ylabel('ACF')
ax.set_title('Autocorrelation: Extremely Slow Decay', fontsize=11, fontweight='bold')
ax.legend()

# Bottom left: Weekly changes histogram
ax = axes[1, 0]
changes = np.diff(treasury_rates)
ax.hist(changes, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='No change')
ax.set_xlabel('Weekly Change (%)')
ax.set_ylabel('Frequency')
ax.set_title(f'Weekly Changes: Mean={np.mean(changes):.4f}, Std={np.std(changes):.4f}', 
             fontsize=11, fontweight='bold')
ax.legend()

# Bottom right: Scatter of y[t] vs y[t-1]
ax = axes[1, 1]
ax.scatter(treasury_rates[:-1], treasury_rates[1:], alpha=0.3, s=10)
min_val, max_val = treasury_rates.min(), treasury_rates.max()
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='y[t]=y[t-1]')
ax.set_xlabel('Rate at week t-1 (%)')
ax.set_ylabel('Rate at week t (%)')
ax.set_title(f'Persistence: Tomorrow ≈ Today (corr={acf_treasury[1]:.3f})', 
             fontsize=11, fontweight='bold', color='red')
ax.legend()

plt.tight_layout()
plt.show()

print("\n★ Key Insight: With ACF(1)=0.995, the theoretical maximum improvement")
print(f"   over persistence is only {(1-0.995)**2 * 100:.4f}%")
print("   → Beating 'predict no change' is nearly impossible!")

---

## Section 2: Stock Market Returns

**What we're mimicking**: Daily S&P 500 returns

**Key characteristics**:
- Returns have VERY LOW persistence (ACF ≈ 0)
- But volatility has HIGH persistence (volatility clustering)
- Fat tails (extreme events more common than normal distribution)

**The insight**: Predicting price LEVELS is easy (just predict yesterday's price). Predicting RETURNS is nearly impossible. But predicting VOLATILITY? That's doable.

In [ ]:
def generate_stock_returns(n_days=2520, vol_persistence=0.9, seed=42):
    """
    Generate synthetic stock returns with GARCH-like volatility clustering.
    
    Key insight:
    - Returns have LOW persistence (φ ≈ 0) — nearly unpredictable
    - Volatility has HIGH persistence (φ ≈ 0.9) — clusters persist
    - Fat tails from t-distribution
    
    Parameters
    ----------
    n_days : int
        Number of daily observations (2520 ≈ 10 years)
    vol_persistence : float
        Persistence of volatility (GARCH beta parameter)
    seed : int
        Random seed
    """
    rng = np.random.default_rng(seed)
    
    # GARCH(1,1)-like volatility dynamics
    vol = np.zeros(n_days)
    vol[0] = 0.01  # 1% daily vol (about 16% annualized)
    
    # GARCH parameters: omega + alpha*r²[t-1] + beta*vol²[t-1]
    omega = 0.00001  # Long-run variance contribution
    alpha = 0.1       # Shock impact
    beta = vol_persistence  # Volatility persistence
    
    returns = np.zeros(n_days)
    
    for t in range(1, n_days):
        # Update volatility (GARCH dynamics)
        vol[t] = np.sqrt(omega + alpha * returns[t-1]**2 + beta * vol[t-1]**2)
        # Generate return with fat tails (t-distribution, df=5)
        returns[t] = vol[t] * rng.standard_t(df=5)
    
    return returns, vol

# Generate 10 years of daily returns
stock_returns, stock_vol = generate_stock_returns(n_days=2520, vol_persistence=0.9, seed=42)

# Compute prices (for illustration)
prices = 100 * np.exp(np.cumsum(stock_returns))  # Start at 100

# Compute ACFs
acf_returns = compute_acf(stock_returns, max_lag=20)
acf_abs_returns = compute_acf(np.abs(stock_returns), max_lag=20)

print(f"Generated {len(stock_returns)} days of synthetic stock returns")
print(f"Mean daily return: {np.mean(stock_returns)*100:.3f}%")
print(f"Daily volatility: {np.std(stock_returns)*100:.2f}%")
print(f"Annualized volatility: {np.std(stock_returns)*np.sqrt(252)*100:.1f}%")
print(f"")
print(f"ACF(1) of returns: {acf_returns[1]:.4f}  ← VERY LOW (unpredictable)")
print(f"ACF(1) of |returns|: {acf_abs_returns[1]:.4f}  ← HIGH (vol clusters)")

In [ ]:
# Visualize stock market dynamics
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

days = np.arange(len(stock_returns))
years = days / 252

# Top left: Price (levels)
ax = axes[0, 0]
ax.plot(years, prices, 'b-', linewidth=0.8)
ax.set_xlabel('Years')
ax.set_ylabel('Price')
ax.set_title('Price Levels: High Persistence (Easy to "Predict")', fontsize=11, fontweight='bold')

# Top right: Returns
ax = axes[0, 1]
ax.plot(years, stock_returns * 100, 'b-', linewidth=0.5, alpha=0.7)
ax.axhline(y=0, color='red', linestyle='--')
ax.set_xlabel('Years')
ax.set_ylabel('Daily Return (%)')
ax.set_title('Returns: NO Persistence (Nearly Random)', fontsize=11, fontweight='bold', color='green')

# Bottom left: Volatility
ax = axes[1, 0]
ax.plot(years, stock_vol * 100 * np.sqrt(252), 'orange', linewidth=1)
ax.set_xlabel('Years')
ax.set_ylabel('Annualized Volatility (%)')
ax.set_title('Volatility: HIGH Persistence (Clusters)', fontsize=11, fontweight='bold', color='red')

# Bottom right: ACF comparison
ax = axes[1, 1]
lags = np.arange(len(acf_returns))
width = 0.35
ax.bar(lags - width/2, acf_returns, width, label='Returns (unpredictable)', color='green', alpha=0.7)
ax.bar(lags + width/2, acf_abs_returns, width, label='|Returns| (vol clustering)', color='orange', alpha=0.7)
ax.axhline(y=0, color='black', linewidth=0.5)
ax.axhline(y=1.96/np.sqrt(len(stock_returns)), color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=-1.96/np.sqrt(len(stock_returns)), color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Lag (days)')
ax.set_ylabel('ACF')
ax.set_title('ACF: Returns vs |Returns|', fontsize=11, fontweight='bold')
ax.legend()

plt.tight_layout()
plt.show()

print("\n★ Key Insight: Don't confuse levels with returns!")
print("   - Predicting PRICE levels is trivial (persistence is ~1.0)")
print("   - Predicting RETURNS is nearly impossible (ACF ≈ 0)")
print("   - Predicting VOLATILITY is feasible (ACF of |returns| ≈ 0.3)")

---

## Section 3: Unemployment Rate

**What we're mimicking**: Monthly U.S. Unemployment Rate

**Key characteristics**:
- High persistence (φ ≈ 0.92)
- Regime changes (recessions cause sudden jumps)
- Asymmetric dynamics (fast up, slow down)

**The challenge**: Persistence is high, AND there are structural breaks. The worst of both worlds for forecasting.

In [ ]:
def generate_unemployment_like(n_months=240, phi=0.92, seed=42):
    """
    Generate synthetic unemployment rate with regime changes.
    
    Features:
    - High persistence within regimes (φ ≈ 0.92)
    - Sudden jumps during recessions
    - Slow decay after shocks (asymmetric)
    - Floor at ~3% (full employment)
    
    Parameters
    ----------
    n_months : int
        Number of monthly observations (240 = 20 years)
    phi : float
        Autoregressive coefficient
    seed : int
        Random seed
    """
    rng = np.random.default_rng(seed)
    
    rates = np.zeros(n_months)
    rates[0] = 5.0  # Start at 5% (normal level)
    
    # Recession timing (roughly every 7-10 years)
    recession_months = [60, 140]  # Two recessions in 20 years
    
    for t in range(1, n_months):
        # Check for recession shock
        shock = 0
        if t in recession_months:
            shock = rng.uniform(2.5, 4.0)  # Jump 2.5-4 percentage points
        
        # Mean-reverting AR(1) with shock
        long_run_mean = 4.5  # Natural rate
        rates[t] = phi * rates[t-1] + (1-phi) * long_run_mean + 0.15 * rng.normal() + shock
        
        # Apply floor (can't go below ~3%)
        rates[t] = max(rates[t], 3.0)
    
    return rates, recession_months

# Generate 20 years of monthly unemployment
unemployment, recessions = generate_unemployment_like(n_months=240, phi=0.92, seed=42)

# Compute ACF
acf_unemp = compute_acf(unemployment, max_lag=24)

print(f"Generated {len(unemployment)} months of synthetic unemployment data")
print(f"Mean rate: {np.mean(unemployment):.1f}%")
print(f"Min/Max: {np.min(unemployment):.1f}% / {np.max(unemployment):.1f}%")
print(f"ACF(1): {acf_unemp[1]:.3f}")
print(f"ACF(12) [1 year]: {acf_unemp[12]:.3f}")

In [ ]:
# Visualize unemployment dynamics
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

months = np.arange(len(unemployment))
years = months / 12

# Left: Time series with recession markers
ax = axes[0]
ax.plot(years, unemployment, 'b-', linewidth=1.5)
for rec in recessions:
    ax.axvline(x=rec/12, color='red', linestyle='--', alpha=0.7, linewidth=2)
ax.axhline(y=4.5, color='green', linestyle=':', alpha=0.7, label='Natural rate')
ax.set_xlabel('Years')
ax.set_ylabel('Unemployment Rate (%)')
ax.set_title('Unemployment: High Persistence + Regime Changes', fontsize=11, fontweight='bold')
ax.legend()

# Annotate recessions
ax.annotate('Recession\nshock', xy=(recessions[0]/12, unemployment[recessions[0]]), 
            xytext=(recessions[0]/12 + 1, unemployment[recessions[0]] + 1),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=9, color='red')

# Middle: ACF
ax = axes[1]
lags = np.arange(len(acf_unemp))
ax.bar(lags, acf_unemp, color='steelblue', alpha=0.7)
ax.axhline(y=0.9, color='red', linestyle='--', label='High persistence')
ax.axhline(y=0, color='black', linewidth=0.5)
ax.set_xlabel('Lag (months)')
ax.set_ylabel('ACF')
ax.set_title('ACF: Slow Decay (φ ≈ 0.92)', fontsize=11, fontweight='bold')
ax.legend()

# Right: Monthly changes histogram
ax = axes[2]
changes = np.diff(unemployment)
ax.hist(changes, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)

# Mark recession jumps
for rec in recessions:
    if rec < len(changes):
        ax.axvline(x=changes[rec-1], color='orange', linestyle='-', linewidth=2, alpha=0.7)

ax.set_xlabel('Monthly Change (%)')
ax.set_ylabel('Frequency')
ax.set_title('Changes: Mostly Small + Rare Large Jumps', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n★ Key Insight: High persistence + structural breaks = worst case")
print("   - Within regimes: persistence makes naive hard to beat")
print("   - Across regimes: breaks invalidate historical patterns")
print("   → Consider regime detection rather than point forecasting")

---

## Section 4: The Persistence Problem

Now let's empirically verify what we've been saying: **high-persistence series are nearly impossible to beat with ML models**.

We'll train a Ridge regression model on each series and compare to the naive forecast.

In [ ]:
def create_lag_features(series, n_lags=5):
    """Create lag features for prediction."""
    n = len(series)
    X = np.column_stack([
        np.concatenate([[np.nan]*lag, series[:-lag]]) 
        for lag in range(1, n_lags + 1)
    ])
    valid = ~np.isnan(X).any(axis=1)
    return X[valid], series[valid]

def evaluate_vs_persistence(series, name, n_lags=5):
    """
    Train a model and compare to persistence baseline.
    Returns MASE and improvement percentage.
    """
    X, y = create_lag_features(series, n_lags=n_lags)
    
    # Train-test split (80-20)
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Train model
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    # Persistence predictions (y[t-1])
    persistence_preds = X_test[:, 0]
    
    # Metrics
    model_mae = mean_absolute_error(y_test, preds)
    persist_mae = mean_absolute_error(y_test, persistence_preds)
    
    # MASE (using training naive errors)
    naive_errors = np.abs(np.diff(y_train))
    naive_mae = np.mean(naive_errors)
    mase = model_mae / naive_mae
    
    improvement = (persist_mae - model_mae) / persist_mae * 100
    
    # ACF(1)
    acf1 = np.corrcoef(series[1:], series[:-1])[0, 1]
    
    return {
        'name': name,
        'acf1': acf1,
        'model_mae': model_mae,
        'persist_mae': persist_mae,
        'mase': mase,
        'improvement': improvement
    }

# Evaluate all three series
results = [
    evaluate_vs_persistence(treasury_rates, "Treasury Rates"),
    evaluate_vs_persistence(unemployment, "Unemployment"),
    evaluate_vs_persistence(stock_returns, "Stock Returns"),
]

print("THE PERSISTENCE PROBLEM: MODEL vs NAIVE FORECAST")
print("=" * 75)
print(f"{'Series':<18} {'ACF(1)':<10} {'Model MAE':<12} {'Naive MAE':<12} {'MASE':<10} {'Improve'}")
print("-" * 75)
for r in results:
    verdict = '✓' if r['mase'] < 1 else '✗'
    print(f"{r['name']:<18} {r['acf1']:<10.3f} {r['model_mae']:<12.4f} {r['persist_mae']:<12.4f} "
          f"{r['mase']:<10.3f} {r['improvement']:>+.1f}% {verdict}")
print("-" * 75)
print("\nMASE < 1 = Model beats naive | MASE ≥ 1 = No skill")

In [ ]:
# Visualize the persistence-difficulty relationship
fig, ax = plt.subplots(figsize=(10, 6))

# Plot each series
acfs = [r['acf1'] for r in results]
mases = [r['mase'] for r in results]
names = [r['name'] for r in results]
colors = ['red' if m >= 1 else 'green' for m in mases]

ax.scatter(acfs, mases, c=colors, s=200, alpha=0.7, edgecolors='black', linewidth=2)

# Add labels
for acf, mase, name in zip(acfs, mases, names):
    ax.annotate(name, (acf, mase), textcoords="offset points", 
                xytext=(10, 5), fontsize=11, fontweight='bold')

# Reference lines
ax.axhline(y=1, color='black', linestyle='--', linewidth=2, label='MASE=1 (no skill)')
ax.axvline(x=0.9, color='red', linestyle=':', alpha=0.7, label='High persistence threshold')

# Theoretical curve (approximate)
phi_range = np.linspace(0, 0.99, 100)
# For very high phi, MASE → 1
theoretical_mase = 1 - 0.5 * (1 - phi_range**2)
ax.plot(phi_range, theoretical_mase, 'gray', linestyle='-', alpha=0.5, 
        label='Approximate lower bound')

ax.set_xlabel('ACF(1) — Persistence Level', fontsize=12)
ax.set_ylabel('MASE — Model Performance', fontsize=12)
ax.set_title('The Persistence Problem: Higher ACF → Harder to Beat Naive', 
             fontsize=13, fontweight='bold')
ax.legend(loc='upper left')
ax.set_xlim(-0.1, 1.05)
ax.set_ylim(0, 1.5)

# Add regions
ax.fill_between([0.9, 1.05], 0, 1.5, color='red', alpha=0.1)
ax.fill_between([-0.1, 0.7], 0, 1.5, color='green', alpha=0.1)

ax.annotate('Hard Zone\n(ACF > 0.9)', xy=(0.95, 0.2), fontsize=10, color='red', 
            ha='center', fontweight='bold')
ax.annotate('Achievable Zone\n(ACF < 0.7)', xy=(0.3, 0.2), fontsize=10, color='green', 
            ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

---

## Section 5: Decision Framework

### Before Building a Model, Ask These Questions

In [ ]:
def diagnose_series(series, name="Your Series"):
    """
    Quick diagnostic for any time series.
    
    Returns ACF(1), persistence classification, and guidance.
    Use this BEFORE building models!
    """
    n = len(series)
    acf1 = np.corrcoef(series[1:], series[:-1])[0, 1]
    
    # Persistence baseline MAE
    persist_errors = np.abs(series[1:] - series[:-1])
    persist_mae = np.mean(persist_errors)
    
    # Theoretical maximum improvement [T1]
    max_improvement = (1 - acf1**2) * 100 if acf1 > 0 else 100
    
    print(f"DIAGNOSTIC: {name}")
    print("=" * 60)
    print(f"Observations:      {n}")
    print(f"ACF(1):            {acf1:.4f}")
    print(f"Persistence MAE:   {persist_mae:.6f}")
    print(f"")
    
    # Classification and guidance
    if acf1 > 0.95:
        level = "EXTREMELY HIGH"
        color = "\033[91m"  # Red
        guidance = [
            "Beating persistence is nearly IMPOSSIBLE.",
            "Consider: Do you even need to predict?",
            "Alternative tasks: Direction, regimes, uncertainty quantification",
            f"Theoretical max improvement: <{max_improvement:.2f}%"
        ]
    elif acf1 > 0.90:
        level = "VERY HIGH"
        color = "\033[93m"  # Yellow
        guidance = [
            "Beating persistence is VERY DIFFICULT.",
            "Use MASE as primary metric (not MAE/RMSE).",
            "Consider move-conditional metrics.",
            f"Theoretical max improvement: <{max_improvement:.1f}%"
        ]
    elif acf1 > 0.70:
        level = "MODERATE"
        color = "\033[93m"  # Yellow
        guidance = [
            "Improvement is POSSIBLE but limited.",
            "Always report MASE alongside MAE.",
            "Expect small but meaningful gains.",
            f"Theoretical max improvement: ~{max_improvement:.0f}%"
        ]
    else:
        level = "LOW"
        color = "\033[92m"  # Green
        guidance = [
            "Standard ML can add SIGNIFICANT value.",
            "MAE/RMSE are meaningful metrics.",
            "Good candidate for model development.",
            f"Theoretical max improvement: ~{max_improvement:.0f}%"
        ]
    
    print(f"Persistence Level: {level}")
    print(f"")
    print("Guidance:")
    for g in guidance:
        print(f"  → {g}")
    
    return {'acf1': acf1, 'persist_mae': persist_mae, 'level': level}

# Demonstrate on each series
print("\n" + "="*60)
diagnose_series(treasury_rates, "Synthetic Treasury Rates")
print("\n" + "="*60)
diagnose_series(stock_returns, "Synthetic Stock Returns")
print("\n" + "="*60)
diagnose_series(unemployment, "Synthetic Unemployment Rate")

### Decision Flowchart

```
Your data → Compute ACF(1)
    │
    ├─ ACF(1) > 0.95 ───────────────────────────────────────────────────┐
    │   │                                                                │
    │   └─ DON'T predict levels                                          │
    │      Consider instead:                                             │
    │      • Direction prediction (up/down)                              │
    │      • Regime detection (normal vs stressed)                       │
    │      • Uncertainty quantification (intervals)                      │
    │                                                                    │
    ├─ 0.7 < ACF(1) < 0.95 ──────────────────────────────────────────┐  │
    │   │                                                             │  │
    │   └─ Use MASE as primary metric                                 │  │
    │      Expect small improvements (2-10%)                          │  │
    │      Consider move-conditional metrics                          │  │
    │                                                                 │  │
    └─ ACF(1) < 0.7 ─────────────────────────────────────────────────┐│  │
        │                                                            ││  │
        └─ Standard ML can add value                                 ││  │
           MAE/RMSE are meaningful                                   ││  │
           Good candidate for modeling                               ││  │
                                                                     ││  │
                                                                     ▼▼  ▼
                                                              Run validation
                                                              with temporalcv
```

---

## Key Takeaways

### 1. Treasury/Interest Rates (ACF ≈ 0.99)
- Persistence is nearly unbeatable
- "Tomorrow = today" is an excellent forecast
- Focus on direction or uncertainty, not point forecasts

### 2. Stock Returns vs Levels
- **Levels** have high persistence (easy to "predict" trivially)
- **Returns** have near-zero persistence (nearly unpredictable)
- **Volatility** has moderate persistence (can be modeled)
- Don't confuse predicting prices with predicting returns!

### 3. Unemployment and Economic Indicators
- High persistence + regime changes = very hard
- Focus on regime detection rather than point forecasting
- Be skeptical of models that "beat" persistence by large margins

### 4. The Rule: Check ACF(1) FIRST

| ACF(1) | What to Expect |
|--------|----------------|
| > 0.95 | Don't predict levels |
| 0.9-0.95 | MASE ≈ 1.0 is normal |
| 0.7-0.9 | Small gains possible |
| < 0.7 | Standard ML works |

---

## Next Steps

Now that you understand WHY time-series validation is critical:

1. **[01_why_temporal_cv.ipynb](01_why_temporal_cv.ipynb)**: Learn to validate correctly with WalkForwardCV
2. **[Feature Engineering Safety Guide](../docs/tutorials/feature_engineering_safety.md)**: Avoid common feature leakage traps
3. **[Metric Selection Guide](../docs/tutorials/metric_selection.md)**: Choose the right metric for your problem

---

*"Always check ACF(1) before building models. If it's above 0.95, ask yourself: is prediction even the right task?"*